In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random
import sys
from itertools import chain

import numpy as np

from bounding_box_classifiers.obama_detector import train_helper

In [ ]:
static_params = {
    "seed": 1,
    "data_dir": "data/bounding-box-obama-detector",
    "use_cuda": True,
    "val_batch_size": 64,
    "epochs": 20,
    "log_interval": 10,
    "save_model": False
}

variable_params = {
    "lr": [0.1, 0.01, 0.001, 0.001],
    "momentum": [0.5, 0.6, 0.7, 0.8, 0.9],
    "batch_size": [32, 64, 128]
}

In [ ]:
used_variable_params = set()

param_space_size = np.prod([len(p) for p in variable_params])
while len(used_params) < param_space_size:
    curr_params = {k: random.choice(v) for k, v in variable_params.items()}
    curr_variable_params = tuple(chain.from_iterable(curr_params.items()))

    if curr_variable_params not in used_variable_params:
        print("Current variable parameters:", curr_variable_params)
        used_variable_params.add(curr_variable_params)
        curr_params.update(static_params)
        train_helper(**curr_params)

# Best results
Both with 62% test accuracy:
```
('--lr', '0.001', '--momentum', '0.9', '--batch-size', '32')  # epoch 2
('--lr', '0.01', '--momentum', '0.6', '--batch-size', '32')  # epoch 5
```

In [10]:
best_params = {
    "lr": 0.01,
    "momentum": 0.6,
    "batch_size": 32
}
best_params.update(static_params)

train_helper(**best_params)

Train Epoch: 1 [0/248 (0%)]	Loss: 0.767255

Test set: Average loss: 4.1701, Accuracy: 33/64 (52%)

Train Epoch: 2 [0/248 (0%)]	Loss: 4.872567

Test set: Average loss: 2.7590, Accuracy: 30/64 (47%)

Train Epoch: 3 [0/248 (0%)]	Loss: 2.006256

Test set: Average loss: 1.6179, Accuracy: 35/64 (55%)

Train Epoch: 4 [0/248 (0%)]	Loss: 1.559603

Test set: Average loss: 1.0682, Accuracy: 37/64 (58%)

Train Epoch: 5 [0/248 (0%)]	Loss: 1.021353

Test set: Average loss: 0.9943, Accuracy: 38/64 (59%)

Train Epoch: 6 [0/248 (0%)]	Loss: 0.670594

Test set: Average loss: 1.0897, Accuracy: 29/64 (45%)

Train Epoch: 7 [0/248 (0%)]	Loss: 0.832940

Test set: Average loss: 0.9243, Accuracy: 34/64 (53%)

Train Epoch: 8 [0/248 (0%)]	Loss: 1.141534

Test set: Average loss: 0.7599, Accuracy: 39/64 (61%)

Train Epoch: 9 [0/248 (0%)]	Loss: 0.803112

Test set: Average loss: 0.7911, Accuracy: 33/64 (52%)

Train Epoch: 10 [0/248 (0%)]	Loss: 0.896224

Test set: Average loss: 0.8298, Accuracy: 31/64 (48%)

Train Epo